In [13]:
from types import SimpleNamespace
%load_ext autoreload
%autoreload 2

from argparse import Namespace
from climpy.utils.tropomi_utils import SENTINEL_DATA_ROOT_PATH, configure_tropomi_credentials, \
    fetch_tropomi_from_wrf_folder, regrid_tropomi_on_wrf_grid_in_batch, derive_information_fraction, \
    add_dates_to_metadata, get_tropomi_configs
import xarray as xr
from pathlib import Path
import glob, os
from datetime import datetime


'''
The goal:
Given the folder with WRF output, download TROPOMI data and calculate corresponding TROPOMI-like diagnostics
'''
# Input args
aqaba_defaults = SimpleNamespace(
    grid_id='AQABA_d01',
    wrf_output_folder_path = '/scratch/osipovs/Data/AirQuality/THOFA/chem_100_v2025.0/',
    wrf_filter_dates = None
)
thofa_defaults = SimpleNamespace(
    grid_id='THOFA_d02',
    wrf_output_folder_path = '/scratch/osipovs/Data/AirQuality/THOFA/inversion/v5/run_srs_ref/',
    # wrf_output_folder_path = '/scratch/osipovs/Data/AirQuality/THOFA/inversion/v5/run_srs_revised/',
    wrf_filter_dates = [datetime(2023, 6, 1), datetime(2023, 6, 25)]  # last wrfout file is not full. Tropomi-like diags break
)
ch4_settings, no2_settings = get_tropomi_configs()
thofa_ch4_config = SimpleNamespace(**{**vars(thofa_defaults), **vars(ch4_settings)})
thofa_no2_config = SimpleNamespace(**{**vars(thofa_defaults), **vars(no2_settings)})
config = thofa_ch4_config
# config = thofa_no2_config

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Getting List of TROPOMI files, CDSE Approach

Alternative way to Download TROPOMI: https://gist.github.com/nicholasbalasus/008b34590fbc55b757fbd879bb64ccc0


In [14]:
meta_df = fetch_tropomi_from_wrf_folder(config.diag_key, config.wrf_output_folder_path)#, wrf_date_format='%Y-%m-%d_%H:%M:%S')
if config.wrf_filter_dates: meta_df = meta_df[meta_df['start_date'].between(config.wrf_filter_dates[0], config.wrf_filter_dates[1])]

# Build the WRF post-processing list to derive TROPOMI-like diagnostics
meta_df['wrfin_file'] = meta_df['start_date'].apply(lambda x: x.strftime('wrfout_d01_%Y-%m-%d_00_00_00'))  # name of the wrf output
meta_df['wrfout_file'] = meta_df['start_date'].apply(lambda x: x.strftime('wrfout_d01_%Y-%m-%d_%H_%M_%S'))  # name for the pp-ed wrf file

# Save list of files for processing
config.meta_fp = config.wrf_output_folder_path + 'pp/tropomi_meta_{}.csv'.format(config.diag_key.lower())
print('Saving metadata to \n{}'.format(config.meta_fp))
Path(config.meta_fp).parent.mkdir(parents=True, exist_ok=True)  # # 2. Extract the parent directory and create it
meta_df.to_csv(config.meta_fp, index=False)#, header=['Name'])

display(meta_df[['Id', 'Name', 'S3Path', 'GeoFootprint']].head(3))
display(meta_df[['Name', 'start_date', 'wrfin_file' , 'wrfout_file']].head(3))

Searching ch4 from 2023-06-01 to 2023-06-25
Bounds: 44.4158935546875, 23.402915954589844 to 56.8514404296875, 33.72188949584961
Saving metadata to 
/scratch/osipovs/Data/AirQuality/THOFA/inversion/v5/run_srs_ref/pp/tropomi_meta_ch4.csv


,Id,Name,S3Path,GeoFootprint
0,3ade07de-d232-414c-a0e8-b1a0e39c50d0,S5P_OFFL_L2__CH4____20230601T081351_20230601T0...,/eodata/Sentinel-5P/TROPOMI/L2__CH4___/2023/06...,"{'type': 'Polygon', 'coordinates': [[[-92.6680..."
1,3fe70b12-42a8-47d8-af08-2c8438ce47f1,S5P_OFFL_L2__CH4____20230601T095521_20230601T1...,/eodata/Sentinel-5P/TROPOMI/L2__CH4___/2023/06...,"{'type': 'Polygon', 'coordinates': [[[-118.042..."
2,f468ab49-02b3-462a-a2f4-fa15eabf8199,S5P_OFFL_L2__CH4____20230602T075450_20230602T0...,/eodata/Sentinel-5P/TROPOMI/L2__CH4___/2023/06...,"{'type': 'Polygon', 'coordinates': [[[-87.9467..."


,Name,start_date,wrfin_file,wrfout_file
0,S5P_OFFL_L2__CH4____20230601T081351_20230601T0...,2023-06-01 08:13:51,wrfout_d01_2023-06-01_00_00_00,wrfout_d01_2023-06-01_08_13_51
1,S5P_OFFL_L2__CH4____20230601T095521_20230601T1...,2023-06-01 09:55:21,wrfout_d01_2023-06-01_00_00_00,wrfout_d01_2023-06-01_09_55_21
2,S5P_OFFL_L2__CH4____20230602T075450_20230602T0...,2023-06-02 07:54:50,wrfout_d01_2023-06-02_00_00_00,wrfout_d01_2023-06-02_07_54_50


In [15]:
import eofetch

sentinel_data_root_path = SENTINEL_DATA_ROOT_PATH
display('Downloading TROPOMI files to: {}'.format(sentinel_data_root_path))
configure_tropomi_credentials()

for index, row in meta_df.iterrows():
    print('downloading {}'.format(row.Name))
    eofetch.download(row.Name, target_directory=sentinel_data_root_path)

print("Done")

'Downloading TROPOMI files to: /project/k10048/osipovs//Data/Copernicus/Sentinel-5P/'

downloading S5P_OFFL_L2__CH4____20230601T081351_20230601T095521_29183_03_020500_20230603T044522.nc
downloading S5P_OFFL_L2__CH4____20230601T095521_20230601T113651_29184_03_020500_20230603T053832.nc
downloading S5P_OFFL_L2__CH4____20230602T075450_20230602T093620_29197_03_020500_20230603T235942.nc
downloading S5P_OFFL_L2__CH4____20230602T093620_20230602T111750_29198_03_020500_20230604T014854.nc
downloading S5P_OFFL_L2__CH4____20230603T073550_20230603T091720_29211_03_020500_20230604T235242.nc
downloading S5P_OFFL_L2__CH4____20230603T091720_20230603T105850_29212_03_020500_20230605T012621.nc
downloading S5P_OFFL_L2__CH4____20230604T085819_20230604T103949_29226_03_020500_20230606T030319.nc
downloading S5P_OFFL_L2__CH4____20230605T083918_20230605T102048_29240_03_020500_20230607T005100.nc
downloading S5P_OFFL_L2__CH4____20230605T102048_20230605T120218_29241_03_020500_20230607T023558.nc
downloading S5P_OFFL_L2__CH4____20230606T082017_20230606T100147_29254_03_020500_20230608T003406.nc
downloadin

# Regrid TROPOMI to WRF grid

In [16]:
regrid_tropomi_on_wrf_grid_in_batch(meta_df, config.grid_id, config.tropomi_key)

Error: Environment variable 'ClimPy' is not set. Assume Workstation /home/osipovs/PycharmProjects/ClimPy/examples/regridding/regrid_tropomi_on_wrf_grid.py

0, Processing S5P_OFFL_L2__CH4____20230601T081351_20230601T095521_29183_03_020500_20230603T044522.nc
  -> Skipping: Output already exists at /project/k10048/osipovs//Data/Copernicus/Sentinel-5P//THOFA_d02//S5P_OFFL_L2__CH4____20230601T081351_20230601T095521_29183_03_020500_20230603T044522.nc

1, Processing S5P_OFFL_L2__CH4____20230601T095521_20230601T113651_29184_03_020500_20230603T053832.nc
  -> Skipping: Output already exists at /project/k10048/osipovs//Data/Copernicus/Sentinel-5P//THOFA_d02//S5P_OFFL_L2__CH4____20230601T095521_20230601T113651_29184_03_020500_20230603T053832.nc

2, Processing S5P_OFFL_L2__CH4____20230602T075450_20230602T093620_29197_03_020500_20230603T235942.nc
  -> Skipping: Output already exists at /project/k10048/osipovs//Data/Copernicus/Sentinel-5P//THOFA_d02//S5P_OFFL_L2__CH4____20230602T075450_20230602T09362

# Derive List of TROPOMI files with good coverage

In [17]:
derive_information_fraction(meta_df, config.tropomi_key, config.grid_id)

# Derive TROPOMI like diagnistics for WRF output

In [18]:
for row in meta_df.itertuples():
    wrf_in = config.wrf_output_folder_path + row.wrfin_file
    wrf_out = config.wrf_output_folder_path + 'pp/tropomi_like_{}/'.format(config.diag_key.lower()) + row.wrfout_file
    tropomi_in=SENTINEL_DATA_ROOT_PATH + '/{}/'.format(config.grid_id) + row.Name

    if os.path.exists(wrf_out):
        print(f"  -> Skipping: Output already exists at {wrf_out}")
        continue

    # %run /home/osipovs/PycharmProjects/ClimPy/climpy/wrf/pp_wrf_like_tropomi_ch4.py --wrf_in={wrf_in} --wrf_out={wrf_out} --tropomi_in={tropomi_in}

    # Pure python version
    args = Namespace(
        wrf_in=wrf_in,
        wrf_out=wrf_out,
        tropomi_in=tropomi_in,
    )

    from climpy.wrf.pp_wrf_like_tropomi_ch4 import pp_wrf_like_tropomi_ch4
    from climpy.wrf.pp_wrf_like_tropomi_no2 import pp_wrf_like_tropomi_no2
    processors = {
        'ch4': pp_wrf_like_tropomi_ch4,
        'no2': pp_wrf_like_tropomi_no2
    }
    pp_wrf_like_tropomi_impl = processors[config.diag_key.lower()]
    pp_wrf_like_tropomi_impl(args)

Will process this WRF:
in /scratch/osipovs/Data/AirQuality/THOFA/inversion/v5/run_srs_ref/wrfout_d01_2023-06-01_00_00_00
out /scratch/osipovs/Data/AirQuality/THOFA/inversion/v5/run_srs_ref/pp/tropomi_like_ch4/wrfout_d01_2023-06-01_08_13_51
tropomi /project/k10048/osipovs//Data/Copernicus/Sentinel-5P//THOFA_d02/S5P_OFFL_L2__CH4____20230601T081351_20230601T095521_29183_03_020500_20230603T044522.nc


/home/osipovs/PycharmProjects/ClimPy/climpy/utils/wrf_utils.py:36: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  binned_mean = s.groupby(bins).mean().values
/home/osipovs/PycharmProjects/ClimPy/climpy/utils/wrf_utils.py:36: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  binned_mean = s.groupby(bins).mean().values


Saving to:
/scratch/osipovs/Data/AirQuality/THOFA/inversion/v5/run_srs_ref/pp/tropomi_like_ch4/wrfout_d01_2023-06-01_08_13_51
Done
Will process this WRF:
in /scratch/osipovs/Data/AirQuality/THOFA/inversion/v5/run_srs_ref/wrfout_d01_2023-06-01_00_00_00
out /scratch/osipovs/Data/AirQuality/THOFA/inversion/v5/run_srs_ref/pp/tropomi_like_ch4/wrfout_d01_2023-06-01_09_55_21
tropomi /project/k10048/osipovs//Data/Copernicus/Sentinel-5P//THOFA_d02/S5P_OFFL_L2__CH4____20230601T095521_20230601T113651_29184_03_020500_20230603T053832.nc


/home/osipovs/PycharmProjects/ClimPy/climpy/utils/wrf_utils.py:36: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  binned_mean = s.groupby(bins).mean().values
/home/osipovs/PycharmProjects/ClimPy/climpy/utils/wrf_utils.py:36: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  binned_mean = s.groupby(bins).mean().values


Saving to:
/scratch/osipovs/Data/AirQuality/THOFA/inversion/v5/run_srs_ref/pp/tropomi_like_ch4/wrfout_d01_2023-06-01_09_55_21
Done
Will process this WRF:
in /scratch/osipovs/Data/AirQuality/THOFA/inversion/v5/run_srs_ref/wrfout_d01_2023-06-02_00_00_00
out /scratch/osipovs/Data/AirQuality/THOFA/inversion/v5/run_srs_ref/pp/tropomi_like_ch4/wrfout_d01_2023-06-02_07_54_50
tropomi /project/k10048/osipovs//Data/Copernicus/Sentinel-5P//THOFA_d02/S5P_OFFL_L2__CH4____20230602T075450_20230602T093620_29197_03_020500_20230603T235942.nc


...

Saving to:
/scratch/osipovs/Data/AirQuality/THOFA/inversion/v5/run_srs_ref/pp/tropomi_like_ch4/wrfout_d01_2023-06-23_09_42_38
Done
Will process this WRF:
in /scratch/osipovs/Data/AirQuality/THOFA/inversion/v5/run_srs_ref/wrfout_d01_2023-06-24_00_00_00
out /scratch/osipovs/Data/AirQuality/THOFA/inversion/v5/run_srs_ref/pp/tropomi_like_ch4/wrfout_d01_2023-06-24_07_42_04
tropomi /project/k10048/osipovs//Data/Copernicus/Sentinel-5P//THOFA_d02/S5P_OFFL_L2__CH4____20230624T074204_20230624T092334_29509_03_020500_20230625T234410.nc


/home/osipovs/PycharmProjects/ClimPy/climpy/utils/wrf_utils.py:36: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  binned_mean = s.groupby(bins).mean().values
/home/osipovs/PycharmProjects/ClimPy/climpy/utils/wrf_utils.py:36: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  binned_mean = s.groupby(bins).mean().values


Saving to:
/scratch/osipovs/Data/AirQuality/THOFA/inversion/v5/run_srs_ref/pp/tropomi_like_ch4/wrfout_d01_2023-06-24_07_42_04
Done
Will process this WRF:
in /scratch/osipovs/Data/AirQuality/THOFA/inversion/v5/run_srs_ref/wrfout_d01_2023-06-24_00_00_00
out /scratch/osipovs/Data/AirQuality/THOFA/inversion/v5/run_srs_ref/pp/tropomi_like_ch4/wrfout_d01_2023-06-24_09_23_34
tropomi /project/k10048/osipovs//Data/Copernicus/Sentinel-5P//THOFA_d02/S5P_OFFL_L2__CH4____20230624T092334_20230624T110504_29510_03_020500_20230626T013203.nc


/home/osipovs/PycharmProjects/ClimPy/climpy/utils/wrf_utils.py:36: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  binned_mean = s.groupby(bins).mean().values
/home/osipovs/PycharmProjects/ClimPy/climpy/utils/wrf_utils.py:36: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  binned_mean = s.groupby(bins).mean().values


Saving to:
/scratch/osipovs/Data/AirQuality/THOFA/inversion/v5/run_srs_ref/pp/tropomi_like_ch4/wrfout_d01_2023-06-24_09_23_34
Done
